In [12]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import polars as pl
import cv2
import msgpack as mp
import msgpack_numpy as mpn
from cv2 import aruco

In [10]:
_webcam_calib_folder = r"C:\Users\CMC\Documents\openposelibs\pose\armbo\recording_programs\test_data\omni_cam_9d0f_may_8_2023\calibration_00"
_webcam_calib_folder = os.path.join(_webcam_calib_folder)
_webcam_calib_pth = os.path.join( _webcam_calib_folder, "webcam_calibration.msgpack")

with open(_webcam_calib_pth, "rb") as f:
    webcam_calib = mp.Unpacker(f, object_hook=mpn.decode)
    _temp = next(webcam_calib)
    _webcam_cam_mat = _temp[0]
    _webcam_dist = _temp[1]
_webcam_video_pth = os.path.join(_webcam_calib_folder, "webcam_color.msgpack")
_webcam_timestamp_pth = os.path.join(_webcam_calib_folder, "webcam_timestamp.msgpack")

with open(os.path.join(_webcam_calib_folder, "webcam_rotmat.msgpack"), "rb") as f:
    webcam_rotmat = mp.Unpacker(f, object_hook=mpn.decode)
    _webcam_rot = next(webcam_rotmat)
    _webcam_org = next(webcam_rotmat)
print("camera matrix", _webcam_cam_mat)
print("distortion", _webcam_dist)



camera matrix [[880.19523452   0.         358.2789884 ]
 [  0.         881.87061144 242.70902376]
 [  0.           0.           1.        ]]
distortion [[ 1.24325941e-01 -1.20330000e-01 -2.99461015e-03 -1.15851300e-03
  -3.99283638e+00]]


In [9]:
_video_pth = r"C:\Users\CMC\Documents\openposelibs\pose\armbo\recording_programs\test_data\omni_cam_9d0f_may_8_2023\sk36_30_random_01\webcam_color.msgpack"

_vid_file = open(_video_pth, "rb")
unpacker = mp.Unpacker(_vid_file, object_hook=mpn.decode)
_video_data = [msg for msg in unpacker]

Aruco detection

In [13]:
ARUCO_PARAMETERS = aruco.DetectorParameters()
DICTIONARY = aruco.getPredefinedDictionary(aruco.DICT_ARUCO_ORIGINAL)
detector = aruco.ArucoDetector(DICTIONARY, ARUCO_PARAMETERS)

In [16]:
corners, ids, rejected_img_points = detector.detectMarkers(_video_data[0])
# draw the marker and show it
# temp_image = aruco.drawDetectedMarkers(temp_image, corners, ids)

aruco.estimatePoseSingleMarkers(corners, 0.05, _webcam_cam_mat, _webcam_dist)

AttributeError: module 'cv2.aruco' has no attribute 'estimatePoseSingleMarkers'

(array([[[418., 467.],
         [419., 501.],
         [385., 500.],
         [384., 466.]]], dtype=float32),)